In [5]:
from pyscf import dft, gto
from xtddft.utils import unit, atom
from xtddft.TDA import TDA
from xtddft.UTDA import UTDA
from xtddft.XTDA import XTDA

# TDA

In [6]:
mol = gto.M(
    atom=atom.n2,  # unit is Angstrom  # in utils/atom.py, give lots of atom
    basis='cc-pvdz',
    unit='A',
    spin=0,
    charge=0,
    verbose=4,
)
mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.conv_tol = 1e-11  # set small to compare pyscf result with my result
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
mf.conv_check = False  # check convergence will run another cycle making result change
mf.kernel()

tda = TDA(mol, mf)
tda.singlet = True  # False for triplet
tda.nstates = 12
tda.savedata = False  # set True will get "uvspec_data.csv" saving excited energy and corresponding oscillator strength
e_eV, os, rs, v = tda.kernel()
tda.analyze()

System: uname_result(system='Linux', node='localhost', release='6.6.87.2-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025', machine='x86_64')  Threads 12
Python 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
numpy 2.4.0  scipy 1.16.3  h5py 3.15.1
Date: Wed Feb 25 10:43:16 2026
PySCF version 2.11.0
PySCF path  /home/whope/miniconda3/envs/stda/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 N      0.000000000000   0.000000000000  -0.558995780000 AA    0.000000000000   0.000000000000  -1.056348928988 Bohr   0.0
[INPUT]  2 N      0.000000000000   0.000000000000   0.558995780000 AA    0.000000000000   0.00

# UTDA

In [7]:
mol = gto.M(
    atom=atom.ch2o_vacuum,  # unit is Angstrom
    basis='cc-pvdz',
    unit='A',
    charge=1,
    spin=1,
    verbose=4
)

# # ground state calculation add solvents
# mf = dft.UKS(mol).SMD()
# # in https://gaussian.com/scrf/ solvents entry, give different eps for different solvents
# # mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# # mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃

mf = dft.UKS(mol)  # if include solvent effect, remember comment this line
mf.xc = 'b3lyp'
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
mf.conv_check = False
mf.kernel()

utda = UTDA(mol, mf)
utda.nstates = 12
utda.savedata = False
e_eV, os, rs, v  = utda.kernel()
utda.analyze()

System: uname_result(system='Linux', node='localhost', release='6.6.87.2-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025', machine='x86_64')  Threads 12
Python 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
numpy 2.4.0  scipy 1.16.3  h5py 3.15.1
Date: Wed Feb 25 10:43:21 2026
PySCF version 2.11.0
PySCF path  /home/whope/miniconda3/envs/stda/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.526270000000   0.000000000000 AA    0.000000000000   0.994506167575   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.979180000000   1.091955000000   0.000000000000 AA    1.850382026652   2.06

# XTDA

In [8]:
mol = gto.M(
    atom=atom.ch2o_vacuum,
    basis='cc-pvdz',
    unit='A',
    charge=1,
    spin=1,
    verbose=4
)
# # add solvents
# mf = dft.ROKS(mol).SMD()
# mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# # mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# # mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃
 
mf = dft.ROKS(mol)
mf.xc = 'b3lyp'
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
mf.kernel()

xtda = XTDA(mol, mf)
xtda.nstates = 12
xtda.savedata = False
# It is recommended to use the spin-orbit basis
xtda.basis = "orbital"  # orbital or tensor
e_eV, os, rs, v = xtda.kernel()
xtda.analyze()
# xtda.basis = "tensor"
# e_eV, v = xtda.kernel()
# xtda.analyze()

System: uname_result(system='Linux', node='localhost', release='6.6.87.2-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025', machine='x86_64')  Threads 12
Python 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
numpy 2.4.0  scipy 1.16.3  h5py 3.15.1
Date: Wed Feb 25 10:43:37 2026
PySCF version 2.11.0
PySCF path  /home/whope/miniconda3/envs/stda/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.526270000000   0.000000000000 AA    0.000000000000   0.994506167575   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.979180000000   1.091955000000   0.000000000000 AA    1.850382026652   2.06